<a href="https://colab.research.google.com/github/pragyansharma24/Gun-Violence-in-United-States/blob/master/Gun_Violence_in_the_United_States.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of Gun Violence in the United States
#### Authored By: Pragyan Sharma, Prerna Patil, Shriya Rao, Sonal Agrawal

---

Over the years, there has been a continuous increase in the number of incidents related to gun violence in the United States. In addition to killing hundreds of Americans, it has forced them to live in a continuous fear of the next shooting. Therefore, in order to understand the magnitude of everyday gun violence, we have analyzed available data to understand:
<ul>
  <li><b>Crime Statistics across United States</b></li>
  <li><b>Crime Statistics across the World</b></li>
</ul>

## Crime Statistics across United States 

### Introduction
According to an article in New York Times, "*More people died from firearm injuries in the United States in 2017 than in any other year since at least 1968, according to new data from the Centers for Disease Control and Prevention*." (Ref: https://www.nytimes.com/2018/12/18/us/gun-deaths.html).

In fact, 2017 witnessed the highest number of mass shootings alone, killing and injuring hundreds of innocent lives. To understand the seriousness of the situation, we plan to visualize each of the mass shooting incidents on the map of the United States.   

In [0]:
import pandas as pd
import numpy as np

### Data Cleaning
The dataset needs some preparation before we start exploring and visualizing the dataset further. Steps to be follwed are:
<ul>
  <li>Renaming columns for efficient data handling</li>
  <li> Creating new features e.g. month and year from date </li>
  <li>Adding latitude and longitude (geolocation) for each of the given locations for maps</li>
  <li>Handling Null values</li>
</ul>

Step1: Reading and merging data for mass shootings from 2013 to 2019

In [0]:
y2013 = pd.read_csv("Mass Shootings year wise/Mass_Shootings_2013.csv")
y2014 = pd.read_csv("Mass Shootings year wise/Mass_Shootings_2014.csv")
y2015 = pd.read_csv("Mass Shootings year wise/Mass_Shootings_2015.csv")
y2016 = pd.read_csv("Mass Shootings year wise/Mass_Shootings_2016.csv")
y2017 = pd.read_csv("Mass Shootings year wise/Mass_Shootings_2017.csv")
y2018 = pd.read_csv("Mass Shootings year wise/Mass_Shootings_2018.csv")
y2019 = pd.read_csv("Mass Shootings year wise/Mass_Shootings_2019.csv")
# combining data for al the years 
mass_shoot = pd.concat([y2013, y2014, y2015, y2016, y2017, y2018, y2019])
mass_shoot.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,"December 31, 2013",New York,Brooklyn,60 Glenmore Ave,0.0,6.0,NaN
1,"December 28, 2013",Alabama,Montgomery,954 Highland Ave,3.0,5.0,NaN
2,"December 26, 2013",Louisiana,Slidell,2144 First St,2.0,6.0,NaN
3,"December 26, 2013",Louisiana,Lockport,313 Tenth St,3.0,3.0,NaN
4,"December 25, 2013",New Jersey,Irvington,Nye Avenue and 21st Street,3.0,2.0,NaN


Step 2: Renaming columns for efficient data handling

In [0]:
mass_shoot.rename(columns={'Incident Date':'Date','City Or County':'City', '# Killed':'num_Killed', '# Injured':'num_Injured'}, inplace=True)

Step 3: Extracting Month and Year from the incident Date column

In [0]:
from datetime import datetime as dt
mass_shoot['Date'] = pd.to_datetime(mass_shoot['Date'])
mass_shoot['Month'] = mass_shoot['Date'].dt.month
mass_shoot['Year'] = mass_shoot['Date'].dt.year

Step 4: Reading data for Latitudes and Longitudes for cities in the United States

In [0]:
cities_location = pd.read_csv("uscitiesv1.5.csv")
# Selecting relevant columns for further analysis
cities_location_sub = cities_location[['city','state_id', 'state_name','lat', 'lng']]
cities_location_sub.head()

,city,state_id,state_name,lat,lng
0,Prairie Ridge,WA,Washington,47.1443,-122.1408
1,Edison,WA,Washington,48.5602,-122.4311
2,Packwood,WA,Washington,46.6085,-121.6702
3,Wautauga Beach,WA,Washington,47.5862,-122.5482
4,Harper,WA,Washington,47.5207,-122.5196


Step 5: Merging mass shootings data with city locations to use with maps. Left join has been used to ensure we have all the data from mass_shoot.

In [0]:
# Merging datasets
mass_shooting_data = mass_shoot.merge(cities_location_sub,how='left', left_on=['City', 'State'], right_on=['city', 'state_name'])
# Dropping extra columns which we won't be using further
mass_shooting_data.drop(columns=['Operations', 'city', 'state_name','state_id'], inplace=True)

Step 6: Removing all the NA values

In [0]:
#Removing NA
mass_shooting_data = mass_shooting_data.dropna()
# Checking number of rows in the combined dataframe
len(mass_shooting_data)

1915

### Final Dataset

In [0]:
mass_shooting_data.head()

,Date,State,City,Address,num_Killed,num_Injured,Month,Year,lat,lng
0,2013-12-31,New York,Brooklyn,60 Glenmore Ave,0.0,6.0,12.0,2013.0,40.6501,-73.9496
1,2013-12-28,Alabama,Montgomery,954 Highland Ave,3.0,5.0,12.0,2013.0,32.3470,-86.2663
2,2013-12-26,Louisiana,Slidell,2144 First St,2.0,6.0,12.0,2013.0,30.2882,-89.7826
3,2013-12-26,Louisiana,Lockport,313 Tenth St,3.0,3.0,12.0,2013.0,29.6418,-90.5376
5,2013-12-25,New York,Medford,33A Cedarhurst Ave,1.0,3.0,12.0,2013.0,40.8220,-72.9859


### Exploration

Number of lives lost in mass shootings by year

In [0]:
mass_shooting_data.groupby('Year')['num_Killed'].sum()\
.reset_index().sort_values(by= 'num_Killed', ascending = False) 

,Year,num_Killed
3,2016.0,411.0
4,2017.0,392.0
2,2015.0,332.0
5,2018.0,328.0
0,2013.0,249.0
1,2014.0,242.0
6,2019.0,130.0


Cities that lost most number of lives in mass shootings

In [0]:
mass_shooting_data.groupby('City')['c','num_Injured'].sum()\
.reset_index().sort_values(by= 'num_Killed', ascending = False)[:5]                      

,City,num_Killed,num_Injured
106,Chicago,82.0,573.0
285,Las Vegas,72.0,488.0
389,Orlando,68.0,102.0
243,Houston,47.0,102.0
466,Saint Louis,37.0,148.0


In [0]:
mass_shooting_data.groupby('State')['num_Killed','num_Injured'].agg(['sum','count','mean','max'])

num_Killed                       num_Injured        \
                            sum count      mean   max         sum count   
State                                                                     
Alabama                    46.0    46  1.000000   5.0       183.0    46   
Alaska                      0.0     1  0.000000   0.0         6.0     1   
Arizona                    47.0    27  1.740741   5.0        89.0    27   
Arkansas                   20.0    18  1.111111   4.0        91.0    18   
California                236.0   223  1.058296  16.0       900.0   223   
Colorado                   23.0    21  1.095238   3.0        76.0    21   
Connecticut                 5.0    14  0.357143   2.0        72.0    14   
Delaware                    9.0    10  0.900000   5.0        38.0    10   
District of Columbia        8.0    30  0.266667   2.0       136.0    30   
Florida                   200.0   129  1.550388  50.0       558.0   129   
Georgia                    84.0    76  1.105263   6.0       286.0    76   
Illinois                  119.0   176  0.676136   6.0       741.0   176   
Indiana                    48.0    52  0.923077   5.0       193.0    52   
Iowa                        4.0     5  0.800000   2.0        18.0     5   
Kansas                     30.0    17  1.764706   5.0        65.0    17   
Kentucky                   30.0    26  1.153846   5.0       102.0    26   
Louisiana                  74.0    80  0.925000   5.0       356.0    80   
Maine                       9.0     2  4.500000   5.0         1.0     2   
Maryland                   52.0    52  1.000000   5.0       198.0    52   
Massachusetts               6.0    15  0.400000   1.0        65.0    15   
Michigan                   54.0    60  0.900000   6.0       242.0    60   
Minnesota                  11.0    16  0.687500   4.0        70.0    16   
Mississippi                49.0    32  1.531250   8.0       107.0    32   
Missouri                   64.0    65  0.984615   4.0       239.0    65   
Montana                     9.0     3  3.000000   5.0         5.0     3   
Nebraska                    6.0     7  0.857143   3.0        32.0     7   
Nevada                     72.0    17  4.235294  59.0       497.0    17   
New Jersey                 34.0    54  0.629630   4.0       216.0    54   
New Mexico                 32.0    13  2.461538   5.0        30.0    13   
New York                   46.0    79  0.582278   5.0       332.0    79   
North Carolina             46.0    53  0.867925   4.0       203.0    53   
Ohio                       74.0    66  1.121212   8.0       263.0    66   
Oklahoma                   22.0    21  1.047619   4.0        74.0    21   
Oregon                     15.0     6  2.500000  10.0        26.0     6   
Pennsylvania               76.0    79  0.962025  11.0       310.0    79   
Rhode Island                1.0     4  0.250000   1.0        16.0     4   
South Carolina             50.0    39  1.282051   9.0       145.0    39   
South Dakota               10.0     2  5.000000   6.0         1.0     2   
Tennessee                  54.0    77  0.701299   6.0       313.0    77   
Texas                     198.0   115  1.721739  10.0       411.0   115   
Utah                        9.0     3  3.000000   5.0         7.0     3   
Virginia                   40.0    46  0.869565   6.0       188.0    46   
Washington                 42.0    21  2.000000   5.0        58.0    21   
West Virginia               5.0     3  1.666667   4.0         9.0     3   
Wisconsin                  15.0    14  1.071429   5.0        47.0    14   

                                        
                           mean    max  
State                                   
Alabama                3.978261   12.0  
Alaska                 6.000000    6.0  
Arizona                3.296296    7.0  
Arkansas               5.055556   25.0  
California             4.035874   19.0  
Colorado               3.619048    9.0  
Connecticut            5.142857   13.0  
Delaware  

### Scope of Analysis
Combined dataset will be used to analyze the following:
<ul>
  <li><b><i>Mass Shootings across United States:</i></b> Geolocation variables will be used to plot the incidents on a map to visualize the distribution of mass shootings in the United States. This will help us in identifying the states and locations where gun violence is relatively higher.</li>
  <ul>
    <li>Key variable of interest: num_Killed, Latitude, Longitude, City</li>
  </ul>
  <li><b><i>Change in Mass Shooting Incidents over the Years:</i></b> The change in number of mass shooting incidents over the years (2013 - 2019) and/or months across United States will be plotted. This will help us in finding any hidden trends in the data.</li>
  <ul>
    <li>Key variable of interest: Year/Months, num_Killed, num_Injured</li>
    </ul>

*** 

## Crime Statistics across the World

### Data Cleaning

Step 1: Reading in data for world crimes

In [0]:
world_crime = pd.read_csv("world_crime.csv")
world_crime.head()

,Country/Territory,ISO code,Source,% of homicides by firearm,Number of homicides by firearm,"Homicide by firearm rate per 100,000 pop",Rank by rate of ownership,Average firearms per 100 people,Average total all civilian firearms
0,Albania,AL,CTS,65.9,56.0,1.76,70.0,8.6,270000.0
1,Algeria,DZ,CTS,4.8,20.0,0.06,78.0,7.6,1900000.0
2,Angola,AO,NaN,NaN,NaN,NaN,34.0,17.3,2800000.0
3,Anguilla,AI,WHO-MDB,24.0,1.0,7.14,NaN,NaN,NaN
4,Argentina,AR,Ministry of Justice,52.0,1198.0,3.02,62.0,10.2,3950000.0


Step 2: Renaming columns for efficient data handling

In [0]:
# Creating a new list of column names
cols = ['Country', 'Country_code', 'Source', '%_homi_gun', 'num_homi_gun', 'homi_gun_per100k', 'rank-rate_owner',\
        'avg_gun_per100', 'avg_tot_all_guns']
# Replacing the existing column names with new names saved in the list 'cols'
world_crime.columns = cols

Step 3: Handling Null values

In [0]:
#Checking columns with 'NA' values
world_crime.isna().sum()

Country              0
Country_code         1
Source              69
%_homi_gun          69
num_homi_gun        69
homi_gun_per100k    69
rank-rate_owner      9
avg_gun_per100       9
avg_tot_all_guns     9
dtype: int64

In [0]:
# Removing all the 'NA' values
world_crime = world_crime.dropna()
len(world_crime)

107

Step 4: Merging the crime data will all the countries and combining with a list of developed countries.

In [0]:
# Reading in data for growth rates for all 'Developed' countries
developed_countries = pd.read_csv("developed.csv")
developed_countries.head()

,cca2,name,area,pop2019,GrowthRate
0,US,United States,9372610,329093.110,1.007119
1,JP,Japan,377930,126854.745,0.997401
2,TR,Turkey,783562,82961.805,1.012756
3,DE,Germany,357114,82438.639,1.001764
4,GB,United Kingdom,242900,66959.016,1.005791


In [0]:
# Checking for NAs
developed_countries.isna().sum()

cca2          0
name          0
area          0
pop2019       0
GrowthRate    0
dtype: int64

Merging world crime data with development index to the fing the position of the United States as compared to other developed countries. We are using inner join because we want to see the data only for developed countries.

In [0]:
# Merging world crimes and growth rates dataset
world_crime_developed = world_crime.merge(developed_countries, how="inner", left_on="Country", right_on="name")
# Removing columns that are not required for the analysis
world_crime_developed.drop(columns = ['Source', 'cca2','name', 'area'], inplace = True)

### Final Dataset

In [0]:
world_crime_developed.head()

,Country,Country_code,%_homi_gun,num_homi_gun,homi_gun_per100k,rank-rate_owner,avg_gun_per100,avg_tot_all_guns,pop2019,GrowthRate
0,Australia,AU,11.5,30.0,0.14,42.0,15.0,3050000.0,25088.636,1.012772
1,Austria,AT,29.5,18.0,0.22,14.0,30.4,2500000.0,8766.201,1.001643
2,Belgium,BE,39.5,70.0,0.68,34.0,17.2,1800000.0,11562.784,1.005589
3,Canada,CA,32.0,173.0,0.51,13.0,30.8,9950000.0,37279.811,1.008823
4,Cyprus,CY,26.3,5.0,0.46,6.0,36.4,275000.0,1198.427,1.007856


### Exploration

Countries as per Gun ownership rate

In [0]:
world_crime_developed.sort_values(by = 'rank-rate_owner')[:5]

,Country,Country_code,%_homi_gun,num_homi_gun,homi_gun_per100k,rank-rate_owner,avg_gun_per100,avg_tot_all_guns,pop2019,GrowthRate
28,United States,US,60.0,9146.0,2.97,1.0,88.8,270000000.0,329093.110,1.007119
26,Switzerland,CH,72.2,57.0,0.77,3.0,45.7,3400000.0,8608.259,1.007517
8,Finland,FI,19.8,24.0,0.45,4.0,45.3,2400000.0,5561.389,1.003405
4,Cyprus,CY,26.3,5.0,0.46,6.0,36.4,275000.0,1198.427,1.007856
25,Sweden,SE,33.9,37.0,0.41,10.0,31.6,2800000.0,10053.135,1.007055


In [0]:
world_crime_developed.sort_values(by = 'avg_tot_all_guns', ascending = False)[:5]

,Country,Country_code,%_homi_gun,num_homi_gun,homi_gun_per100k,rank-rate_owner,avg_gun_per100,avg_tot_all_guns,pop2019,GrowthRate
28,United States,US,60.0,9146.0,2.97,1.0,88.8,270000000.0,329093.110,1.007119
10,Germany,DE,26.3,158.0,0.19,15.0,30.3,25000000.0,82438.639,1.001764
9,France,FR,9.6,35.0,0.06,12.0,31.2,19000000.0,65480.710,1.003793
3,Canada,CA,32.0,173.0,0.51,13.0,30.8,9950000.0,37279.811,1.008823
27,Turkey,TR,16.9,535.0,0.77,52.0,12.5,9000000.0,82961.805,1.012756


### Scope of Analysis
The combined dataset can be used to analyze the following: 

<ul>
  <li><b><i>Homicides by firearm per 1 million people</i></b>
This visualization will showcase developed countries according to firearms related homicides. The argument we are trying to present here is that gun availability is a major risk factor for homicides in developed nations. </li>
  <ul> <li>Key variables of interest - Country, homi_gun_per100k </li> </ul>
<li>
  <b><i>Population vs Ownership of firearms </i></b> In this graph we will plot the population of various countries as well as the ownership of firearms in those countries. This visualization will show us a comparison of the number of people living in a country versus the number of civilians who own firearms in those countries. </li>
  <ul> <li>Key variables of interest: Country, pop2019, avg_tot_all_guns </li></ul>
  
  <li>
    <b><i>Gun related deaths per 100k people vs guns per 100 people</i></b> This visualization will be a scatter plot gun related homicides against gun possession across developed countries. This visualization will help us identify relationship between gun ownership and gun related deaths across developed nations.
    <ul><li>Key variables of interest: Country, homi_gun_per100k, avg_gun_per100</li></ul>
  </li>

*** 